# Восстановление пропущенных пробелов

Задача — разработать модель или алгоритм, который принимает на вход текст без пробелов и возвращает восстановленный текст с правильными пробелами и позициями, где они были пропущены

In [1]:
#Импорт библиотек
import pandas as pd
import re
import math
from wordfreq import word_frequency

### Обработка данных

Запятые в тексте неверно разделяют данные, поэтому необходимо предобработка данных: в строках все запятые после первой не будем принимать за разделители.

In [2]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as f:
        lines = f.readlines()

    text_list = []

    #До 1 запятой - индекс, после - текст
    for line in lines:
        comma_index = line.find(",")
        text_list.append(line[comma_index+1:].strip().lower())

    df = pd.DataFrame({"text_no_spaces": text_list})
    return df

Загрузим данные, первую строку, не содержащую данных, удалим

In [3]:
df = read_data("dataset_1937770_3.txt")
df.drop(df.index[0], axis=0, inplace=True)
pd.concat([df.head(3), df.tail(3)])

,text_no_spaces
1,куплюайфон14про
2,ищудомвподмосковье
3,сдаюквартирусмебельюитехникой
1003,весна-скоровырастеттрава.
1004,"весна-выпосмотрите,каккрасиво."
1005,весна-гдемояголова?


Данные больше не нуждаются в обработке

### Разработка модели

Идея написана в README

In [4]:
STOPWORDS = {"в", "и", "к", "с", "а", "я", "у", "о", "на"}
CLOSERS = set(".,!?;:)]}»")
OPENERS = set("([{«")
QUOTES = set("\"'“”«»")

Вычисляем score, используя библиотеку wordfreq. Вычисляем штрафы и бонусы

In [5]:
def score_word(word):
    freq = word_frequency(word, "ru") #вероятность встретить слово
    score = math.log(freq + 1e-9) #правильнее вычислять сумму логарифмов, а не произведение вероятностей

    if freq == 0:
        score -= 20 #сильный штраф за неизвестное
    if len(word) == 1 and word not in STOPWORDS:
        score -= 5 #штраф за одиночные символы кроме STOPWORDS
    if len(word) > 15 and freq == 0:
        score -= 30 #штраф за длинный кусок

    if "-" in word:
        score += 1 #бонус за дефис
    if re.match(r"^\d+[а-яa-z]+$", word):
        score += 2 #бонус для числа и единицы измерения
    
    return score

С помощью динамического программирования разбиваем строку на слова так, чтобы суммарный score был максимален

In [6]:
def segment_dp(text):
    n = len(text)
    dp = [(-1e9, None) for _ in range(n+1)] #инициализация таблицы dp, где dp[i] хранит набор слов и его score
    dp[0] = (0, [])

    for i in range(1, n + 1):
        best = (-1e9, [])
        for j in range(max(0, i - 20), i):
            word = text[j:i]
            sc = score_word(word) #вычисляем score для подстроки word
            cand_score = dp[j][0] + sc #и вычисляем суммарный score
            if cand_score > best[0]:
                best = (cand_score, dp[j][1] + [word])
        dp[i] = best

    return dp[n][1]

Делаем разбиение по спец. символам: знаки препинания, цифры, латиница

In [7]:
def add_segment(text):
    #Разбиваем по знакам препинания
    parts = re.split(r"([.,!?;:()\[\]{}\"«»])", text)

    result = []
    for part in parts:

        #Пропускаем пробелы
        if not part.strip(): 
            continue

        #Добавляем знак пунктуации как отдельный токен
        if re.fullmatch(r"[.,!?;:()\[\]{}\"«»]", part):
            result.append(part)
            continue

        #Разбиваем по очевидным границам, где стык кириллицы и латиницы, цифры и буквы и т.д.
        subparts = re.split(
            r"(?<=\d)(?=[а-яa-zA-Z])|(?<=[а-яa-zA-Z])(?=\d)|(?<=[a-zA-Z])(?=[а-яА-Я])|(?<=[а-яА-Я])(?=[a-zA-Z])",
            part,
        )

        #Для каждой части sp вызываем segment_dp
        for sp in subparts:
            if sp.strip():
                result.extend(segment_dp(sp))

    return result

Собираем строку из токенов

In [8]:
def join_tokens(tokens):
    out = ""
    for i, t in enumerate(tokens): #проходимся по каждому токену
        if t in CLOSERS: #закрывающую пунктуацию присоединяем к левому содержимому
            out += t
            if i + 1 < len(tokens) and tokens[i + 1] not in CLOSERS | QUOTES: #если после не идет друого закрвыющего знака, ставим пробел
                out += " "
        elif t in OPENERS or t in QUOTES: #открывающую пунктуацию присоединяем к правому содержимому
            if out and not out.endswith(" "): #если нужен пробел
                out += " "
            out += t
        elif t == "-":
            if i > 0 and i + 1 < len(tokens):
                #Если слева и справа от "-" буквы, то это одно слово, пробелы не нужны
                if re.match(r"\w", tokens[i - 1]) and re.match(r"\w", tokens[i + 1]):
                    out += "-"
                else:
                    out += " - "
            else:
                out += "-"
        else:
            #Обычное слово
            if out and not out.endswith((" ", "(", "[", "{", "«", "“")):
                out += " "
            out += t
    return out.strip()

Создание списка с индексами пробелов. Текст с пробелами далее генерируется с помощью созданной функции add_segment

In [9]:
def get_space_indices(original_text, true_text):
    space_indexes = []
    prim = true_text
    while prim.find(' ') != -1: #пока есть пробелы
        space_ind = prim.find(' ')
        space_indexes.append(space_ind)
        prim = prim[:space_ind] + prim[space_ind+1:] #удаляем пробел, чтобы индексы не путались

    return space_indexes


Обрабатываем все тексты и сохраняем результаты

In [17]:
results = []

for idx, text in enumerate(df["text_no_spaces"]):
    tokens = add_segment(text) #генерируем токены
    restored_text = join_tokens(tokens) #создаем текст с пробелами
    space_positions = get_space_indices(text, restored_text) #создаем список с пробелами

    #Записываем в слоаврь
    results.append({
        "id": idx, #индекс
        "original_text": text, #изначальный текст без пробелов
        "restored_text": restored_text, #текст с пробелами
        "predicted_positions": space_positions #индексы пробелов
    })

#Создаем DataFrame с результатами
results_df = pd.DataFrame(results)
print(results_df.head())

   id                  original_text                       restored_text  \
0   0                куплюайфон14про                  куплю айфон 14 про   
1   1             ищудомвподмосковье               ищу дом в подмосковье   
2   2  сдаюквартирусмебельюитехникой  сдаю квартиру с мебелью и техникой   
3   3     новыйдивандоставканедорого       новый диван доставка недорого   
4   4                отдамдаромкошку                   отдам даром кошку   

   predicted_positions  
0          [5, 10, 12]  
1            [3, 6, 7]  
2  [4, 12, 13, 20, 21]  
3          [5, 10, 18]  
4              [5, 10]  


### Оценка модели

Не пригодилась, все автоматизировано

In [11]:
'''
F1_list = []
for index, row in results_df.iterrows():
    #precision = |предсказанные ∩ истинные| / |предсказанные|
    precision = len(set(row["space_positions"]) & set(row["true_positions"])) / len(row["space_positions"])
    
    #recall = |предсказанные ∩ истинные| / |истинные|
    recall = len(set(row["space_positions"]) & set(row["true_positions"])) / len(row["true_positions"])
    try:
        F1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        F1 = 0
    F1_list.append(F1)

F1_average = sum(F1_list) / len(F1_list)
print(F1_average)
'''
print()

### Файл с результатом

DataFrame submission в соответствии с заданием

In [18]:
submission = results_df.drop(["original_text", "restored_text"], axis=1, inplace=False) #оставляем id и predicted_positions
submission.head()

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"


In [19]:
submission.to_csv("submission.csv")